## Project 1: House Price Prediction
## 1. Problem Definition
### Predict house prices based on feature like size, location, and amenities. 
### We will use linear regression model first

In [ ]:
# We will use Boston data which is available in scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# set random seed for reproducibility
np.random.seed(42)
# Load the california housing data
housing = fetch_california_housing()
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['PRICE'] = housing.target
#display(data)
print(f"Dataset shape: {data.shape}")
print(f"\nFeature names: {housing.feature_names}")
print(f"\nFirst 5 rows of our dataset: {data.head()}")
print(f"\nStatistical summary: {data.describe()}")

# Missing values -  NO MISSING VALUE
print(f"Missing values in each column: {data.isnull().sum()}")

# Visualize the distribution of house prices -HISTOGRAM
plt.figure(figsize=(10,6))
sns.histplot(data['PRICE'], kde=True, )
plt.title('Distribution of House Prices')
plt.xlabel('Prices (x $100k)')
plt.ylabel('Frequency')
plt.show()

#Correlation matrix to see relationship between features
correlation_matrix = data.corr()
plt.figure(figsize=(12,10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

#Split the data into features (X) and target (y)
X=data.drop('PRICE', axis=1)
y=data['PRICE']
# Split into tarining and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

#Train the model on training data
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred = linear_model.predict(X_test)
#Evaluate the model using test data
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Linear Regression Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# Let's see the coefficients to understand feature importance
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': linear_model.coef_
})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)
print("\nFeature Coefficients:")
print(coefficients)

### We will use LAZYPREDICT to see the top models

In [ ]:
# Using LazyPredict to try multiple models quickly
import lazypredict
from lazypredict.Supervised import LazyRegressor

# Initialize LazyRegressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and evaluate multiple models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# Display performance comparison of various models
print("\nLazyPredict Model Comparison:")
print(models)

# Let's visualize the top performing models
plt.figure(figsize=(12, 6))
models_r2 = models.sort_values(by='R-Squared', ascending=False).head(10)
sns.barplot(x=models_r2.index, y=models_r2['R-Squared'])
plt.xticks(rotation=90)
plt.title('Top 10 Models by R-Squared')
plt.tight_layout()
plt.show()

### We identified XGBoost is the top performing model, so we will train and hyperparameter tune it.

In [ ]:
# Import XGB
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
#Initialize the XGBoost regressor
xgb_model = xgb.XGBRegressor(random=42)
# check basic performance with cross-validation
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='r2')
print(f"XGBoost cross-validation R2 score: {cv_scores}")
print(f"Mean CV R2 score: {cv_scores.mean():.4f}")


In [ ]:
# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
xgb_pred = xgb_model.predict(X_test)

# Evaluate performance
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_pred)

In [ ]:
plt.figure(figsize=(10,6))
xgb.plot_importance(xgb_model)
plt.title('XGBoost Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
# Hyperparameter tuning
param_grid = {
    'n_estimators': [100,200],
    'max_depth': [3,5,7],
    'learning_rate': [0.1,0.05]
}

grid_search = GridSearchCV(
    estimator= xgb.XGBRegressor(random=42),
    param_grid=param_grid,
    cv = 3,
    scoring='r2',
    verbose=1
)

# Fiting the grid search and getting result
grid_search.fit(X_train, y_train)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R2 Score: {grid_search.best_score_}")

In [ ]:
# Final model with best parameters
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train, y_train)
# Final Evaluation
final_pred = best_xgb.predict(X_test)
final_r2 = r2_score(y_test, final_pred)
final_rmse = np.sqrt(mean_squared_error(y_test, final_pred))
#Print final results
print(f"Root Mean Squared Error: {final_rmse:.4f}")
print(f"R² Score: {final_r2:.4f}")

In [ ]:
# VISULIZE
# Create a DataFrame for easier plotting
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': final_pred
})
# Calculate prediction errors
results_df['Error'] = results_df['Actual'] - results_df['Predicted']

# Visualize predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(results_df['Actual'], results_df['Predicted'], alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Prices ($100,000s)')
plt.ylabel('Predicted Prices ($100,000s)')
plt.title('Actual vs Predicted House Prices')
plt.tight_layout()
plt.show()

# Visualize error distribution
plt.figure(figsize=(10, 6))
sns.histplot(results_df['Error'], kde=True)
plt.xlabel('Prediction Error ($100,000s)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.axvline(x=0, color='r', linestyle='--')
plt.tight_layout()
plt.show()

# Visualize errors vs predicted values (to check for patterns)
plt.figure(figsize=(10, 6))
plt.scatter(results_df['Predicted'], results_df['Error'], alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Prices ($100,000s)')
plt.ylabel('Prediction Error ($100,000s)')
plt.title('Prediction Errors vs Predicted Values')
plt.tight_layout()
plt.show()


In [ ]:
import shap
import lime 
from lime import lime_tabular

# AS we already have our trained model XGBoost model (best_xgb) and our test data X_test

# 1. SHAP (SHapley Additive exPlanations)
#create a SHAP explainer for our model
explainer = shap.TreeExplainer(best_xgb)
shap_values = explainer.shap_values(X_test)


In [ ]:
# Visualize SHAP values for a single prediction
plt.figure(figsize=(12,6))
shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:],
                feature_names=X_test.columns, matplotlib=True)
plt.title("SHAP Force plot for a single prediction")
plt.tight_layout()
plt.show()

# Summary plot showing impact of all features
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test)
plt.title("SHAP Summary Plot")
plt.tight_layout()
plt.show()

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from nonconformist.cp import IcpRegressor
from nonconformist.base import RegressorAdapter
from nonconformist.nc import RegressorNc, AbsErrorErrFunc

# Assuming we have our data from Project 1
# Convert pandas DataFrames to NumPy arrays
X_np = X.values
y_np = y.values

# Split data into training, calibration, and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)
X_train, X_calib, y_train, y_calib = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Calibration set: {X_calib.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Create an XGBoost model (our best model from Project 1)
model = xgb.XGBRegressor(random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make point predictions on the test set
point_predictions = model.predict(X_test)

# Calculate baseline metrics
baseline_rmse = np.sqrt(np.mean((y_test - point_predictions) ** 2))
print(f"Baseline RMSE without conformal prediction: {baseline_rmse:.4f}")

# Create a nonconformist adapter that wraps our model
adapter = RegressorAdapter(model)

# Create a nonconformity function - we use absolute error
nc = RegressorNc(adapter, AbsErrorErrFunc())

# Create an inductive conformal predictor (ICP)
icp = IcpRegressor(nc)

# Calibrate the conformal predictor using the calibration set
icp.fit(X_calib, y_calib)

# Generate prediction intervals at 90% confidence
significance = 0.1  # This means 90% confidence (1 - 0.1)
prediction_intervals = icp.predict(X_test, significance=significance)

# Calculate the width of each interval
interval_widths = prediction_intervals[:, 1] - prediction_intervals[:, 0]
average_width = np.mean(interval_widths)

# Calculate coverage (percentage of true values inside intervals)
coverage = np.mean((y_test >= prediction_intervals[:, 0]) & 
                    (y_test <= prediction_intervals[:, 1]))

print(f"90% Prediction Interval - Average width: {average_width:.4f}")
print(f"90% Prediction Interval - Empirical coverage: {coverage:.4f}")

# Display sample predictions with intervals
print("\nSample predictions with intervals:")
for i in range(5):
    print(f"Example {i+1}:")
    print(f"True value: {y_test[i]:.4f}")
    print(f"Predicted value: {point_predictions[i]:.4f}")
    print(f"90% prediction interval: [{prediction_intervals[i, 0]:.4f}, {prediction_intervals[i, 1]:.4f}]")
    print(f"Interval width: {interval_widths[i]:.4f}")
    print(f"True value within interval: {(y_test[i] >= prediction_intervals[i, 0]) and (y_test[i] <= prediction_intervals[i, 1])}")
    print()